In [31]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [32]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [22]:
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password = "Post23!Fnord",
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [35]:
query = "SELECT * from worldhappinessreport_2021"

df = pd.read_sql(query, con=engine)

In [36]:
df.head()

,country_name,regional_indicator,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,expby_log_gdp_per_cap,expby_social_support,expby_healty_life_exp,expby_freedom,expby_generosity,expby_percept_corrupt,distopia_plus_resid
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


In [38]:
list(df)

['country_name',
 'regional_indicator',
 'ladder_score',
 'stand_err',
 'up_whisker',
 'low_whisker',
 'log_gdp_per_cap',
 'social_support',
 'healty_life_exp',
 'freedom',
 'generosity',
 'percept_corrupt',
 'ladder_score_distopia',
 'expby_log_gdp_per_cap',
 'expby_social_support',
 'expby_healty_life_exp',
 'expby_freedom',
 'expby_generosity',
 'expby_percept_corrupt',
 'distopia_plus_resid']

In [47]:
columns = [
    "regional_indicator", "stand_err", "up_whisker", "low_whisker",
    "log_gdp_per_cap", "social_support", "healty_life_exp", "freedom",
    "generosity", "percept_corrupt", "ladder_score_distopia", "expby_log_gdp_per_cap",
    "expby_social_support", "expby_healty_life_exp", "expby_freedom", "expby_generosity",
    "expby_percept_corrupt", "distopia_plus_resid"
]

target = ["ladder_score"]

In [49]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df = df.astype({'ladder_score':'int'})

df.head()

,country_name,regional_indicator,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,expby_log_gdp_per_cap,expby_social_support,expby_healty_life_exp,expby_freedom,expby_generosity,expby_percept_corrupt,distopia_plus_resid
0,Finland,Western Europe,7,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


In [50]:
# Create our features
X = pd.get_dummies(df.drop(columns="ladder_score"))


# Create our target
y = df["ladder_score"]

In [51]:
X.describe()

,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,...,regional_indicator_Central and Eastern Europe,regional_indicator_Commonwealth of Independent States,regional_indicator_East Asia,regional_indicator_Latin America and Caribbean,regional_indicator_Middle East and North Africa,regional_indicator_North America and ANZ,regional_indicator_South Asia,regional_indicator_Southeast Asia,regional_indicator_Sub-Saharan Africa,regional_indicator_Western Europe
count,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,1.490000e+02,...,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000
mean,0.058752,5.648007,5.417631,9.432208,0.814745,64.992799,0.791597,-0.015134,0.727450,2.430000e+00,...,0.114094,0.080537,0.040268,0.134228,0.114094,0.026846,0.046980,0.060403,0.241611,0.140940
std,0.022001,1.054330,1.094879,1.158601,0.114889,6.762043,0.113332,0.150657,0.179226,5.347044e-15,...,0.318998,0.273040,0.197251,0.342047,0.318998,0.162177,0.212309,0.239035,0.429503,0.349133
min,0.026000,2.596000,2.449000,6.635000,0.463000,48.478000,0.382000,-0.288000,0.082000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.043000,4.991000,4.706000,8.541000,0.750000,59.802000,0.718000,-0.126000,0.667000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.054000,5.625000,5.413000,9.569000,0.832000,66.603000,0.804000,-0.036000,0.781000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.070000,6.344000,6.128000,10.421000,0.905000,69.600000,0.877000,0.079000,0.845000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.173000,7.904000,7.780000,11.647000,0.983000,76.953000,0.970000,0.542000,0.939000,2.430000e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
# Check the balance of our target values
y.value_counts()

5    49
4    36
6    35
7    17
3    11
2     1
Name: ladder_score, dtype: int64

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({6: 24, 4: 27, 3: 9, 5: 36, 7: 14, 2: 1})

In [54]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [55]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.7106449106449106

In [56]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[ 0,  2,  0,  0,  0],
       [ 0,  8,  1,  0,  0],
       [ 0,  2, 11,  0,  0],
       [ 0,  0,  1,  9,  1],
       [ 0,  0,  0,  0,  3]], dtype=int64)

In [57]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          3       0.00      0.00      1.00      0.00      0.00      0.00         2
          4       0.67      0.89      0.86      0.76      0.88      0.77         9
          5       0.85      0.85      0.92      0.85      0.88      0.77        13
          6       1.00      0.82      1.00      0.90      0.90      0.80        11
          7       0.75      1.00      0.97      0.86      0.99      0.97         3

avg / total       0.80      0.82      0.94      0.80      0.85      0.76        38



In [58]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1766702063964013, 'up_whisker'),
 (0.13434439279531768, 'low_whisker'),
 (0.05575446870788725, 'expby_log_gdp_per_cap'),
 (0.05280373581212532, 'social_support'),
 (0.052241679490766864, 'log_gdp_per_cap'),
 (0.04949232003240749, 'expby_healty_life_exp'),
 (0.04540131285614552, 'expby_freedom'),
 (0.04340832600034489, 'freedom'),
 (0.04313361221071999, 'healty_life_exp'),
 (0.041894551026841205, 'expby_social_support'),
 (0.03660720142934333, 'distopia_plus_resid'),
 (0.03621481434086348, 'expby_percept_corrupt'),
 (0.03528852388628428, 'stand_err'),
 (0.03456955871663419, 'percept_corrupt'),
 (0.021700957793525436, 'expby_generosity'),
 (0.021041387258017456, 'generosity'),
 (0.013533403532675563, 'regional_indicator_Western Europe'),
 (0.008820966408934277, 'regional_indicator_Middle East and North Africa'),
 (0.006693441726934331, 'regional_indicator_Central and Eastern Europe'),
 (0.00618356203496464, 'regional_indicator_Sub-Saharan Africa'),
 (0.0036029823250918803, 'country_n